In [43]:
import geopandas as gpd
import pandas as pd

In [44]:
# https://www.geospatial.jp/ckan/dataset/biodic-mesh
gdf_mesh = gpd.read_file('mesh1.zip!mesh1.shp').loc[:, ["Name", "geometry"]] # EPSG:4326
gdf_mesh

,Name,geometry
0,30365015,"POLYGON ((136.06250 20.42500, 136.06250 20.433..."
1,30365016,"POLYGON ((136.07500 20.42500, 136.07500 20.433..."
2,36225734,"POLYGON ((122.92500 24.44167, 122.92500 24.450..."
3,36225744,"POLYGON ((122.92500 24.45000, 122.92500 24.458..."
4,36225735,"POLYGON ((122.93750 24.44167, 122.93750 24.450..."
...,...,...
388328,68482720,"POLYGON ((148.87500 45.51667, 148.87500 45.525..."
388329,68482730,"POLYGON ((148.87500 45.52500, 148.87500 45.533..."
388330,68482701,"POLYGON ((148.88750 45.50000, 148.88750 45.508..."
388331,68482711,"POLYGON ((148.88750 45.50833, 148.88750 45.516..."


In [45]:
# http://cellmap.rukihena.com/mls44011.json
df_MLS_1 = pd.read_json('mls44011.json').fillna('')
df_MLS_1

,lat,lon,area,cell,unit,range,samples,created,updated
0,35.673958,139.447787,1,51200257,1,23,2,1651282432,1651282432
1,34.684203,135.764690,1,67932160,7,74,3,1651307728,1651307728
2,34.722659,135.726660,1,68311395,17,76,4,1646717700,1646717700
3,34.708910,135.711737,1,68445141,15,37,2,1646717700,1646717700
4,34.620415,135.686617,1,68665311,38,71,3,1649312459,1649312459
...,...,...,...,...,...,...,...,...,...
107673,26.211672,127.674478,65440,219391760,495,97,2,1637397058,1637397058
107674,26.252926,127.710943,65440,219394837,500,27,2,1649833550,1649833550
107675,26.207604,127.698940,65440,219397890,493,187,2,1650355595,1653655222
107676,26.271765,127.728284,65440,219398668,503,59,3,1649833550,1649833550


In [46]:
# ECI(cell)範囲指定
# https://mls.js2hgw.com/wiki/?用語集#a19d1c53

# PCI(unit)範囲指定しマクロセルを抽出
# https://mls.js2hgw.com/wiki/?用語集#k2c8cab6

df_MLS_2 = df_MLS_1.query('(226492172 >= cell >= 4194305) & (389 >= unit >= 0) & (area < 49154)').copy()
df_MLS_2

,lat,lon,area,cell,unit,range,samples,created,updated
0,35.673958,139.447787,1,51200257,1,23,2,1651282432,1651282432
1,34.684203,135.764690,1,67932160,7,74,3,1651307728,1651307728
2,34.722659,135.726660,1,68311395,17,76,4,1646717700,1646717700
3,34.708910,135.711737,1,68445141,15,37,2,1646717700,1646717700
4,34.620415,135.686617,1,68665311,38,71,3,1649312459,1649312459
...,...,...,...,...,...,...,...,...,...
105470,26.206256,127.650858,49153,219125507,364,157,17,1615531790,1638168571
105471,26.205694,127.651278,49153,219125508,82,154,5,1615531790,1635547686
105472,26.206295,127.650935,49153,219125509,339,184,24,1615531790,1638168571
105473,26.205718,127.651105,49153,219125510,11,291,12,1615531790,1638168571


In [47]:
# 特殊TAC と 都道府県別TAC のみを抽出(移動局・Casaは除く)
df_MLS_3 = df_MLS_2.query('area == [111, 112, 113, 114, 211, 212, 213, 214, 311, 312, 313, 314, 1025, 1026, 1027, 1028, 1029, 2049, 2050, 3073, 3074, 4097, 4098, 5121, 5122, 6145, 6146, 7169, 7170, 7171, 8193, 8194, 8195, 8196, 9217, 9218, 10241, 10242, 11265, 11266, 11267, 11268, 12289, 12290, 12291, 12292, 13313, 13314, 13315, 13316, 15361, 15362, 15363, 15364, 15365, 15366, 16385, 16386, 16387, 17409, 18433, 19457, 20481, 21505, 21506, 21507, 22529, 22530, 22531, 23553, 23554, 23555, 23556, 24577, 24578, 24579, 24580, 25511, 25512, 25513, 25514, 25601, 25602, 26625, 26626, 27649, 27650, 28673, 28674, 28675, 29697, 29698, 29699, 29700, 30721, 30722, 31745, 31746, 32769, 33411, 33793, 33794, 34817, 34818, 34819, 35841, 35842, 35843, 36865, 36866, 37889, 38913, 39937, 39938, 40961, 40962, 41985, 41986, 41987, 41988, 41989, 41990, 43009, 43010, 44033, 44034, 45057, 45058, 45059, 46081, 46082, 47105, 47106, 48129, 48130, 49153, 49154]')
df_MLS_3

,lat,lon,area,cell,unit,range,samples,created,updated
6,35.595048,139.671186,111,71304961,152,195,12,1641783356,1644559786
7,35.593327,139.668826,111,71304962,193,2156,15,1613968756,1652514056
8,35.594231,139.669523,111,71304963,150,17414,138,1606905428,1654414776
9,35.592671,139.675619,111,71304964,141,407,55,1610653938,1647706142
10,35.594507,139.674061,111,71304966,95,285,36,1610653938,1647706142
...,...,...,...,...,...,...,...,...,...
105470,26.206256,127.650858,49153,219125507,364,157,17,1615531790,1638168571
105471,26.205694,127.651278,49153,219125508,82,154,5,1615531790,1635547686
105472,26.206295,127.650935,49153,219125509,339,184,24,1615531790,1638168571
105473,26.205718,127.651105,49153,219125510,11,291,12,1615531790,1638168571


In [48]:
# 緯度経度をgeometryに変換
gdf_MLS = gpd.GeoDataFrame(df_MLS_3, geometry = gpd.points_from_xy(df_MLS_3.lon, df_MLS_3.lat), crs="EPSG:4326")
gdf_MLS

,lat,lon,area,cell,unit,range,samples,created,updated,geometry
6,35.595048,139.671186,111,71304961,152,195,12,1641783356,1644559786,POINT (139.67119 35.59505)
7,35.593327,139.668826,111,71304962,193,2156,15,1613968756,1652514056,POINT (139.66883 35.59333)
8,35.594231,139.669523,111,71304963,150,17414,138,1606905428,1654414776,POINT (139.66952 35.59423)
9,35.592671,139.675619,111,71304964,141,407,55,1610653938,1647706142,POINT (139.67562 35.59267)
10,35.594507,139.674061,111,71304966,95,285,36,1610653938,1647706142,POINT (139.67406 35.59451)
...,...,...,...,...,...,...,...,...,...,...
105470,26.206256,127.650858,49153,219125507,364,157,17,1615531790,1638168571,POINT (127.65086 26.20626)
105471,26.205694,127.651278,49153,219125508,82,154,5,1615531790,1635547686,POINT (127.65128 26.20569)
105472,26.206295,127.650935,49153,219125509,339,184,24,1615531790,1638168571,POINT (127.65094 26.20630)
105473,26.205718,127.651105,49153,219125510,11,291,12,1615531790,1638168571,POINT (127.65111 26.20572)


In [49]:
mergedGdf = gpd.sjoin(gdf_mesh, gdf_MLS[['area', 'samples', 'geometry']])
mergedGdf = mergedGdf[['Name', 'area', 'samples', 'geometry']].copy()
mergedGdf.sort_values('Name', inplace=True)
mergedGdf.reset_index(drop=True, inplace=True)
mergedGdf

,Name,area,samples,geometry
0,39271543,49153,4,"POLYGON ((127.66250 26.11667, 127.66250 26.125..."
1,39271553,49153,9,"POLYGON ((127.66250 26.12500, 127.66250 26.133..."
2,39271553,49153,31,"POLYGON ((127.66250 26.12500, 127.66250 26.133..."
3,39271553,49153,10,"POLYGON ((127.66250 26.12500, 127.66250 26.133..."
4,39271553,49153,18,"POLYGON ((127.66250 26.12500, 127.66250 26.133..."
...,...,...,...,...
95738,68411521,1028,91,"POLYGON ((141.63750 45.43333, 141.63750 45.441..."
95739,68411523,1028,31,"POLYGON ((141.66250 45.43333, 141.66250 45.441..."
95740,68411531,1028,18,"POLYGON ((141.63750 45.44167, 141.63750 45.450..."
95741,68411617,1028,228,"POLYGON ((141.83750 45.42500, 141.83750 45.433..."


In [50]:
df0 = pd.DataFrame(mergedGdf)
df1 = df0.loc[df0.groupby('Name')['samples'].idxmax()]
df1.sort_values('Name', inplace=True)
df1.reset_index(drop=True, inplace=True)
df1

,Name,area,samples,geometry
0,39271543,49153,4,"POLYGON ((127.66250 26.11667, 127.66250 26.125..."
1,39271553,49153,31,"POLYGON ((127.66250 26.12500, 127.66250 26.133..."
2,39271554,49153,18,"POLYGON ((127.67500 26.12500, 127.67500 26.133..."
3,39271563,49153,6,"POLYGON ((127.66250 26.13333, 127.66250 26.141..."
4,39271565,49153,24,"POLYGON ((127.68750 26.13333, 127.68750 26.141..."
...,...,...,...,...
39882,68411521,1028,91,"POLYGON ((141.63750 45.43333, 141.63750 45.441..."
39883,68411523,1028,31,"POLYGON ((141.66250 45.43333, 141.66250 45.441..."
39884,68411531,1028,18,"POLYGON ((141.63750 45.44167, 141.63750 45.450..."
39885,68411617,1028,228,"POLYGON ((141.83750 45.42500, 141.83750 45.433..."


In [51]:
TAC_dic = {111: '京浜 内陸', 112: '京浜 湾岸', 113: '城西', 114: '埼京線～西武池袋線', 211: '山手線 東京～大塚', 212: '山手線 有楽町～恵比寿', 213: '山手線 池袋～渋谷', 214: '山手線 内側', 311: '大宮+埼京線の東', 312: '京浜 湾岸', 313: '城東の南', 314: '城東の北', 1025: '北海道', 1026: '北海道', 1027: '北海道', 1028: '北海道', 1029: '北海道', 2049: '青森県', 2050: '青森県', 3073: '岩手県', 3074: '岩手県', 4097: '宮城県', 4098: '宮城県', 5121: '秋田県', 5122: '秋田県', 6145: '山形県', 6146: '山形県', 7169: '福島県', 7170: '福島県', 7171: '福島県', 8193: '茨城県', 8194: '茨城県', 8195: '茨城県', 8196: '茨城県', 9217: '栃木県', 9218: '栃木県', 10241: '群馬県', 10242: '群馬県', 11265: '埼玉県', 11266: '埼玉県', 11267: '埼玉県', 11268: '埼玉県', 12289: '千葉県', 12290: '千葉県', 12291: '千葉県', 12292: '千葉県', 13313: '東京都', 13314: '東京都', 13315: '東京都', 13316: '東京都', 15361: '神奈川県', 15362: '神奈川県', 15363: '神奈川県', 15364: '神奈川県', 15365: '神奈川県', 15366: '神奈川県', 16385: '新潟県', 16386: '新潟県', 16387: '新潟県', 17409: '富山県', 18433: '石川県', 19457: '福井県', 20481: '山梨県', 21505: '長野県', 21506: '長野県', 21507: '長野県', 22529: '岐阜県', 22530: '岐阜県', 22531: '岐阜県', 23553: '静岡県', 23554: '静岡県', 23555: '静岡県', 23556: '静岡県', 24577: '愛知県', 24578: '愛知県', 24579: '愛知県', 24580: '愛知県', 25511: '大阪 北', 25512: '大阪 南', 25513: '神戸', 25514: '京都', 25601: '三重県', 25602: '三重県', 26625: '滋賀県', 26626: '滋賀県', 27649: '京都府', 27650: '京都府', 28673: '大阪府', 28674: '大阪府', 28675: '大阪府', 29697: '兵庫県', 29698: '兵庫県', 29699: '兵庫県', 29700: '兵庫県', 30721: '奈良県', 30722: '奈良県', 31745: '和歌山県', 31746: '和歌山県', 32769: '鳥取県', 33411: '名古屋', 33793: '島根県', 33794: '島根県', 34817: '岡山県', 34818: '岡山県', 34819: '岡山県', 35841: '広島県', 35842: '広島県', 35843: '広島県', 36865: '山口県', 36866: '山口県', 37889: '徳島県', 38913: '香川県', 39937: '愛媛県', 39938: '愛媛県', 40961: '高知県', 40962: '高知県', 41985: '福岡県', 41986: '福岡県', 41987: '福岡県', 41988: '福岡県', 41989: '福岡県', 41990: '福岡県', 43009: '佐賀県', 43010: '佐賀県', 44033: '長崎県', 44034: '長崎県', 45057: '熊本県', 45058: '熊本県', 45059: '熊本県', 46081: '大分県', 46082: '大分県', 47105: '宮崎県', 47106: '宮崎県', 48129: '鹿児島県', 48130: '鹿児島県', 49153: '沖縄県', 49154: '沖縄県'}

df1['所在地'] = df1['area'].apply(lambda x: TAC_dic[x])

In [52]:
gdf_TAC = gpd.GeoDataFrame(df1[['area', '所在地', 'geometry']], crs='EPSG:4326')

gdf_TAC = gdf_TAC.dissolve(by = 'area', as_index = False)

gdf_TAC

,area,geometry,所在地
0,111,"POLYGON ((139.57500 35.48333, 139.57500 35.491...",京浜 内陸
1,112,"MULTIPOLYGON (((139.63750 35.40000, 139.62500 ...",京浜 湾岸
2,113,"POLYGON ((139.58750 35.65833, 139.58750 35.666...",城西
3,114,"MULTIPOLYGON (((139.57500 35.75000, 139.56250 ...",埼京線～西武池袋線
4,211,"MULTIPOLYGON (((139.75000 35.68333, 139.76250 ...",山手線 東京～大塚
...,...,...,...
131,47105,"MULTIPOLYGON (((131.37500 31.97500, 131.37500 ...",宮崎県
132,47106,"MULTIPOLYGON (((131.08750 31.69167, 131.07500 ...",宮崎県
133,48129,"MULTIPOLYGON (((130.20000 31.84167, 130.20000 ...",鹿児島県
134,48130,"MULTIPOLYGON (((130.53750 31.55833, 130.53750 ...",鹿児島県


In [53]:
import folium
import branca.colormap as cm

In [54]:
colormap = cm.StepColormap(
    colors=['#FF0000', '#FF9932', '#FFFF00', '#65FF32', '#008000', '#32CCFF', '#3265FF', '#FF32FF', '#ffffff', '#656565'],
    vmin=0,
    vmax=10,
    index=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
)

def ReturnNumber(nums):
    str_nums = str(nums)
    return int(str(int(str_nums[-1]) + int(str_nums[-2]))[-1])

In [55]:
# ベースマップ
map = folium.Map(
    tiles = None,
    location = [34.5496419, 135.7788521],
    zoom_start = 9,
    control_scale = True
)

# 国土地理院(白地図)
folium.raster_layers.TileLayer(
    'https://cyberjapandata.gsi.go.jp/xyz/blank/{z}/{x}/{y}.png',
    name = "国土地理院(白地図)",
    attr = "<a href='https://maps.gsi.go.jp/development/ichiran.html'>国土地理院</a>"
).add_to(map)

# Googleマップ標準
folium.raster_layers.TileLayer(
    'https://{s}.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
    subdomains = ['mt0','mt1','mt2','mt3'],
    name = "Google Map",
    attr = "<a href='https://developers.google.com/maps/documentation' target='_blank'>Google Map</a>"
).add_to(map)

In [56]:
folium.GeoJson(
    data = gdf_TAC.to_json(),
    style_function = lambda x:{
        "fillColor": colormap(ReturnNumber(x['properties']['area'])),
        'fillOpacity': 0.7,
        "stroke": False
    },
    name = 'TAC',
    show = True,
    tooltip = folium.features.GeoJsonTooltip(
        fields=['area', '所在地'],
        aliases=['TAC', '所在地']
    )
).add_to(map)

In [57]:
# レイヤーコントロール
folium.LayerControl().add_to(map)

In [58]:
map

In [59]:
map.save('map_tac_mesh.html')